In [1]:
from sklearnex import patch_sklearn 
patch_sklearn()

Intel(R) Extension for Scikit-learn* enabled (https://github.com/intel/scikit-learn-intelex)


In [2]:
import pandas as pd
from sklearn.feature_selection import RFECV

# G-Mean
from imblearn.metrics import  geometric_mean_score
from sklearn.metrics import make_scorer

# Balanced Bagging
from imblearn.ensemble import BalancedBaggingClassifier
from sklearn.tree import DecisionTreeClassifier

from sklearn.preprocessing import LabelEncoder

import numpy as np
from sklearn.model_selection import StratifiedKFold

from sklearn.multiclass import OneVsRestClassifier
from sklearn.multiclass import OneVsOneClassifier

from sklearn.metrics import multilabel_confusion_matrix

from sklearn.metrics import classification_report

from sklearn.svm import LinearSVC, SVC
from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import StandardScaler

from skopt import BayesSearchCV
from sklearn.model_selection import StratifiedKFold
from skopt.space import Categorical, Integer, Real


In [3]:
df = pd.read_csv("test_poc2.csv")

#df = df.sample(frac=1)

df[' Label'].value_counts()

C:\Users\icarus\AppData\Local\Temp\ipykernel_11536\2735813485.py:1: DtypeWarning: Columns (84) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv("test_poc2.csv")


BENIGN     7000
UDP        7000
Syn        7000
Portmap    7000
NetBIOS    7000
MSSQL      7000
LDAP       7000
UDPLag     1873
Name:  Label, dtype: int64

In [4]:
df.head()
#print(df.columns)

,Flow ID,Source IP,Source Port,Destination IP,Destination Port,Protocol,Timestamp,Flow Duration,Total Fwd Packets,Total Backward Packets,...,Active Std,Active Max,Active Min,Idle Mean,Idle Std,Idle Max,Idle Min,SimillarHTTP,Inbound,Label
0,255.255.255.255-0.0.0.0-67-68-17,0.0.0.0,68,255.255.255.255,67,17,01:45.0,72345107,10,0,...,0.000000,9930155,9930155,10402492.0,3.104260e+06,14240466,7286320,0,0,BENIGN
1,192.168.50.9-87.250.250.119-39712-443-6,192.168.50.9,39712,87.250.250.119,443,6,01:46.1,54870391,12,15,...,211.277779,109454,108901,10130842.4,1.989103e+02,10131112,10130604,0,0,BENIGN
2,172.105.219.195-192.168.50.9-443-55460-6,192.168.50.9,55460,172.105.219.195,443,6,01:47.7,65237686,115,56,...,1010.871241,2177143,2174510,7830396.0,2.192265e+03,7833895,7828340,0,0,BENIGN
3,172.105.219.195-192.168.50.9-443-55762-6,192.168.50.9,55762,172.105.219.195,443,6,01:48.7,10156940,9,4,...,0.000000,186703,186703,9818943.0,0.000000e+00,9818943,9818943,0,0,BENIGN
4,185.11.128.203-192.168.50.9-443-45916-6,192.168.50.9,45916,185.11.128.203,443,6,01:49.7,63066401,34,22,...,774949.479300,1898318,58,9689801.5,1.071652e+06,10127877,7502301,0,0,BENIGN


In [5]:
drop_cols = [
    "Flow ID",
    " Source IP",
    " Destination IP",
    " Timestamp",
    " Source Port",
    " Destination Port"
]

df.drop(drop_cols, axis=1, inplace=True)
df = df.replace([np.inf, -np.inf], np.nan).dropna(axis=0)

for col in df.columns:
    if col != " Label":
        df[col] = pd.to_numeric(df[col], errors='coerce')

df = df.dropna(axis=0)

In [6]:
df.head()

,Protocol,Flow Duration,Total Fwd Packets,Total Backward Packets,Total Length of Fwd Packets,Total Length of Bwd Packets,Fwd Packet Length Max,Fwd Packet Length Min,Fwd Packet Length Mean,Fwd Packet Length Std,...,Active Std,Active Max,Active Min,Idle Mean,Idle Std,Idle Max,Idle Min,SimillarHTTP,Inbound,Label
0,17,72345107,10,0,3000,0,300,300,300.000000,0.000000,...,0.000000,9930155,9930155,10402492.0,3.104260e+06,14240466,7286320,0.0,0,BENIGN
1,6,54870391,12,15,0,62,0,0,0.000000,0.000000,...,211.277779,109454,108901,10130842.4,1.989103e+02,10131112,10130604,0.0,0,BENIGN
2,6,65237686,115,56,38572,10976,752,0,335.408696,348.228345,...,1010.871241,2177143,2174510,7830396.0,2.192265e+03,7833895,7828340,0.0,0,BENIGN
3,6,10156940,9,4,3008,408,752,0,334.222222,396.338800,...,0.000000,186703,186703,9818943.0,0.000000e+00,9818943,9818943,0.0,0,BENIGN
4,6,63066401,34,22,2155,1454,462,0,63.382353,148.817972,...,774949.479300,1898318,58,9689801.5,1.071652e+06,10127877,7502301,0.0,0,BENIGN


In [7]:
gmean_scorer = make_scorer(geometric_mean_score)
y = df.loc[:, " Label"].copy()
X = df.drop(labels=" Label", axis=1)

In [8]:
y.value_counts()

UDP        6992
LDAP       6985
MSSQL      6745
BENIGN     6728
Portmap    6708
NetBIOS    6683
Syn        6599
UDPLag     1873
Name:  Label, dtype: int64

In [9]:
le = LabelEncoder()

y_encoded = le.fit_transform(y)


np.unique(y_encoded)

array([0, 1, 2, 3, 4, 5, 6, 7])

In [9]:
clf = make_pipeline(StandardScaler(),LinearSVC())

In [10]:
rfecv = RFECV(
            estimator=clf,
            scoring=gmean_scorer,
            n_jobs=5,
            importance_getter="named_steps.linearsvc.coef_",
        )
    
rfecv.fit(X, y_encoded)



# Pull features and relevant metrics
scores = rfecv.grid_scores_.tolist()
features = X.columns[rfecv.support_].tolist()
ranks = rfecv.ranking_.tolist()

# Assemble results into dictionary.
outDict = {
    "features": features,
    "feature importances": scores,
    "ranking": ranks
        
}

In [11]:
print(len(outDict['features']))
print(outDict['features'])

62
[' Protocol', ' Flow Duration', ' Total Fwd Packets', ' Total Backward Packets', 'Total Length of Fwd Packets', ' Total Length of Bwd Packets', ' Fwd Packet Length Max', ' Fwd Packet Length Min', ' Fwd Packet Length Mean', ' Fwd Packet Length Std', 'Bwd Packet Length Max', ' Bwd Packet Length Min', ' Bwd Packet Length Mean', ' Bwd Packet Length Std', 'Flow Bytes/s', ' Flow Packets/s', ' Flow IAT Mean', ' Flow IAT Std', ' Flow IAT Max', ' Flow IAT Min', 'Fwd IAT Total', ' Fwd IAT Mean', ' Fwd IAT Std', ' Fwd IAT Max', ' Fwd IAT Min', 'Bwd IAT Total', ' Bwd IAT Mean', ' Bwd IAT Std', ' Bwd IAT Max', 'Fwd PSH Flags', 'Fwd Packets/s', ' Bwd Packets/s', ' Min Packet Length', ' Max Packet Length', ' Packet Length Mean', ' Packet Length Std', ' Packet Length Variance', ' RST Flag Count', ' ACK Flag Count', ' URG Flag Count', ' CWE Flag Count', ' Down/Up Ratio', ' Average Packet Size', ' Avg Fwd Segment Size', ' Avg Bwd Segment Size', 'Subflow Fwd Packets', ' Subflow Fwd Bytes', ' Subflow B

In [10]:
spaces = {    
        "estimator": make_pipeline(StandardScaler(), OneVsRestClassifier(SVC(kernel="rbf", random_state=42))),
        "search_spaces": {
            "onevsrestclassifier__estimator__C": Real(0.00001, 10000, prior="log-uniform"),
            "onevsrestclassifier__estimator__gamma": Real(0.000001, 100, prior="log-uniform"),
        }, 
}


In [11]:
#current_X = X.loc[:, outDict["features"]]
current_X = X.copy()
y_encoded = pd.Series(y_encoded) 

In [12]:
k = 5
kf = StratifiedKFold(n_splits=k, random_state=42, shuffle=True)

counter = 0

for train_index, test_index in kf.split(current_X, y_encoded):
            opt = BayesSearchCV(estimator=spaces["estimator"],search_spaces=spaces["search_spaces"],n_iter=50,scoring=gmean_scorer,cv=5,n_jobs=5)
            X_train, X_test = current_X.iloc[train_index,:], current_X.iloc[test_index,:]
            y_train, y_test = y_encoded.iloc[train_index], y_encoded.iloc[test_index]

            #predicted_values = OneVsOneClassifier(clf).fit(X_train, y_train).predict(X_test)
            opt.fit(X_train, y_train)
            predicted_values = opt.predict(X_test)
            #cm = multilabel_confusion_matrix(y_test, predicted_values)

            counter +=1
            print(f"Fold {counter}:\n")
            #print(cm)
            print(opt.best_params_)
            print(classification_report(y_test, predicted_values))

c:\Users\icarus\AppData\Local\Programs\Python\Python310\lib\site-packages\skopt\optimizer\optimizer.py:449: UserWarning: The objective has been evaluated at this point before.
  warnings.warn("The objective has been evaluated "
c:\Users\icarus\AppData\Local\Programs\Python\Python310\lib\site-packages\skopt\optimizer\optimizer.py:449: UserWarning: The objective has been evaluated at this point before.
  warnings.warn("The objective has been evaluated "
c:\Users\icarus\AppData\Local\Programs\Python\Python310\lib\site-packages\skopt\optimizer\optimizer.py:449: UserWarning: The objective has been evaluated at this point before.
  warnings.warn("The objective has been evaluated "


Fold 1:

OrderedDict([('onevsrestclassifier__estimator__C', 10000.0), ('onevsrestclassifier__estimator__gamma', 13.449702237080436)])
              precision    recall  f1-score   support

           0       0.86      1.00      0.92      1346
           1       0.97      0.98      0.98      1397
           2       0.83      0.94      0.88      1349
           3       0.55      0.87      0.68      1337
           4       0.74      0.23      0.35      1341
           5       0.99      0.86      0.92      1320
           6       0.94      0.92      0.93      1398
           7       0.81      0.77      0.79       375

    accuracy                           0.83      9863
   macro avg       0.84      0.82      0.80      9863
weighted avg       0.84      0.83      0.81      9863



c:\Users\icarus\AppData\Local\Programs\Python\Python310\lib\site-packages\skopt\optimizer\optimizer.py:449: UserWarning: The objective has been evaluated at this point before.
  warnings.warn("The objective has been evaluated "
c:\Users\icarus\AppData\Local\Programs\Python\Python310\lib\site-packages\skopt\optimizer\optimizer.py:449: UserWarning: The objective has been evaluated at this point before.
  warnings.warn("The objective has been evaluated "


Fold 2:

OrderedDict([('onevsrestclassifier__estimator__C', 117.26718290624045), ('onevsrestclassifier__estimator__gamma', 0.21401601127581532)])
              precision    recall  f1-score   support

           0       0.98      1.00      0.99      1346
           1       0.98      0.98      0.98      1397
           2       0.90      0.93      0.92      1349
           3       0.56      0.93      0.70      1337
           4       0.70      0.23      0.35      1341
           5       1.00      0.99      0.99      1320
           6       0.91      0.84      0.87      1399
           7       0.69      0.75      0.72       374

    accuracy                           0.84      9863
   macro avg       0.84      0.83      0.81      9863
weighted avg       0.85      0.84      0.82      9863



c:\Users\icarus\AppData\Local\Programs\Python\Python310\lib\site-packages\joblib\externals\loky\process_executor.py:702: UserWarning: A worker stopped while some jobs were given to the executor. This can be caused by a too short worker timeout or by a memory leak.
  warnings.warn(
c:\Users\icarus\AppData\Local\Programs\Python\Python310\lib\site-packages\skopt\optimizer\optimizer.py:449: UserWarning: The objective has been evaluated at this point before.
  warnings.warn("The objective has been evaluated "
c:\Users\icarus\AppData\Local\Programs\Python\Python310\lib\site-packages\skopt\optimizer\optimizer.py:449: UserWarning: The objective has been evaluated at this point before.
  warnings.warn("The objective has been evaluated "
